In [1]:
import rasterio as rio
from rasterio.enums import Resampling
from rasterio import Affine, MemoryFile
import numpy as np
import os
import fiona
import rasterio.mask

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
def resample(i_n,ip,op,res):
#     i_n=os.listdir(ip)# here i_n is file name and ip is folder(input), op is output folder name
#     for i in i_n:
        with rio.open(os.path.join(ip,i_n)) as raster:
            x,y= raster.res
            d_x=x/res
            d_y=y/res
            scale=d_x
            t = raster.transform

    # rescale the metadata
            transform = Affine(t.a / scale, t.b, t.c, t.d, t.e / scale, t.f)
            height = raster.height * scale
            width = raster.width * scale

            profile = raster.profile
            profile.update(transform=transform, driver='GTiff', height=height, width=width)

            data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
                out_shape=(raster.count, int(height), int(width)),
                resampling=Resampling.bilinear,
                )
        o_n=i_n+'_resample.tif'
        raster.close()
        with rio.open(os.path.join(op,o_n),'w',**profile) as dataset:
            dataset.write(data)
        dataset.close()


In [6]:
def crop_sh(ip,op):
#ip is input folder and op is output folder   
  
        with fiona.open(r"D:\IndianWGC84\WGC84india.shp") as shapefile:
            shapes = [feature["geometry"] for feature in shapefile]
        fn=os.listdir(ip)
        for i in fn:
            with rio.open(os.path.join(ip,i),'r+')as src:
                out_image, out_transform = rasterio.mask.mask(src, shapes, crop= True)
                out_meta = src.meta
            out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
            o_n=i.split(".tif")[0]+'_crop.tif'
            with rio.open(os.path.join(op,o_n), "w", **out_meta) as dest:
                dest.write(out_image)

In [4]:
def crop_shf(shf,ipf,op):
# shf is the shape file ipf is the input file and op is output folder   
  
        with fiona.open(shf) as shapefile:
            shapes = [feature["geometry"] for feature in shapefile]

        with rio.open(ipf,'r+')as src:
            out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            out_meta = src.meta
        out_meta.update({"driver": "GTiff",
             "height": out_image.shape[1],
             "width": out_image.shape[2],
             "transform": out_transform})
        n=ipf.split("\\")[-1]
        o_n=n.split('.jp2')[0]+'_crop.tif'

        with rio.open(os.path.join(op,o_n), "w", **out_meta) as dest:
            dest.write(out_image)

In [6]:
def splitband(ip,op_412,op_443):#input folder opfolders
    i_n=os.listdir(ip)
    for i in i_n:
        with rio.open(os.path.join(ip,i)) as srs:
            pr=srs.profile
            pr.update(count='1')
            l_412=srs.read(1)
            l_443=srs.read(2)
        o_n412=i+"412.tif"
        o_n443=i+"443.tif"
        with rio.open(os.path.join(op_412,o_n412),'w',**pr) as w412:
            w412.write(l_412,1)
        with rio.open(os.path.join(op_443,o_n443),'w',**pr) as w443:
            w443.write(l_443,1)
        
#

In [9]:
ip=r"F:\ocm-tiff\16may2018"
op_412=r"F:\ocmtiff_412\16may2018"
op_443=r"F:\ocmtif_443\16may2018"
splitband(ip,op_412,op_443)

In [10]:
in_412=r"F:\ocmtiff_412\16may2018"
in_443=r"F:\ocmtif_443\16may2018"
out412=r"F:\ocm_resampled412\16may2018"
out443=r"F:\ocm_resampled443\16may2018"
i_n412= os.listdir(in_412)
i_n443=os.listdir(in_443)
for i in i_n412:
    resample(i,in_412,out412)
for i in i_n443:
    resample(i,in_443,out443)

    

In [3]:

with rio.open(r"D:\npk\20181003\crop\R10m\T16TCM_20181003T164121_B02_crop.tif") as r:
    print(r.res)

(10.0, 10.0)


In [5]:
op=r"F:\mtech_project_data\cropped\15jun2018"
ip=r"F:\mtech_project_data\mosaiced\15jun2018"
crop_sh(ip,op)


In [ ]:
files=[r"D:\npk\20181003\crop\R10m\T16TCM_20181003T164121_B02_crop.tif",r"D:\npk\20181003\crop\R10m\T16TCM_20181003T164121_B03_crop.tif",r"D:\npk\20181003\crop\R10m\T16TCM_20181003T164121_B04_crop.tif",r"D:\npk\20181003\crop\R10m\T16TCM_20181003T164121_B08_crop.tif"]